## User Satisfactory Analysis

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import os 
import plotly.graph_objects as go
import plotly.express as px          
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances_argmin_min

In [17]:
os.chdir('..')


In [18]:
from database.database_connection import DatabaseConnection
from database.database import DatabaseProcessor
from src.utils import check_missing_values, filter_numerical_columns, check_duplicates, remove_outliers
from src.plot_utils import plot_count, plot_boxplot

In [19]:
database_connection = DatabaseConnection()
database_processor = DatabaseProcessor()

In [20]:
connection = database_connection.connect()


Successfully connected to the database.


In [21]:
query ='SELECT * FROM xdr_data'
df = database_connection.execute_query(query)

In [22]:
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [29]:
# get the engagmeent cluster and the experience cluster

df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Calculate session duration
df['Session Duration (ms)'] = (df['End'] - df['Start']).dt.total_seconds() * 1000

# Calculate total traffic for each session (download + upload)
df['Total Traffic (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

# Group by 'MSISDN/Number' to calculate metrics for each user
user_engagement = df.groupby('MSISDN/Number').agg({
    'Start': 'count',                           # Sessions frequency
    'Session Duration (ms)': 'mean',           # Average session duration
    'Total Traffic (Bytes)': 'sum'             # Total traffic for all sessions
})

# Rename columns for better readability
user_engagement = user_engagement.rename(columns={
    'Start': 'Sessions Frequency',
    'Session Duration (ms)': 'Average Session Duration (ms)',
    'Total Traffic (Bytes)': 'Total Traffic (Bytes)'
})

# Display the result
print(user_engagement)

               Sessions Frequency  Average Session Duration (ms)  \
MSISDN/Number                                                      
3.360100e+10                    1                    116760000.0   
3.360100e+10                    1                    181260000.0   
3.360100e+10                    1                    135000000.0   
3.360101e+10                    1                     49920000.0   
3.360101e+10                    2                     18600000.0   
...                           ...                            ...   
3.379000e+10                    1                      8760000.0   
3.379000e+10                    1                    141000000.0   
3.197021e+12                    1                    877380000.0   
3.370000e+14                    1                    253020000.0   
8.823971e+14                    1                    869820000.0   

               Total Traffic (Bytes)  
MSISDN/Number                         
3.360100e+10            8.786906e+08 

In [ ]:
# normalize the engagment metrics 
scaler = StandardScaler()
normalized_engagment = scaler.fit_transform(user_engagement)

# run K-means clustering with k=3
Kmeans = KMeans(n_clusters=3,random_state=42)
# user_engagement['CLuster'] = Kmeans.fit_predict(normalized_engagment)

print(user_engagement)

               Sessions Frequency  Average Session Duration (ms)  \
MSISDN/Number                                                      
3.360100e+10                    1                    116760000.0   
3.360100e+10                    1                    181260000.0   
3.360100e+10                    1                    135000000.0   
3.360101e+10                    1                     49920000.0   
3.360101e+10                    2                     18600000.0   
...                           ...                            ...   
3.379000e+10                    1                      8760000.0   
3.379000e+10                    1                    141000000.0   
3.197021e+12                    1                    877380000.0   
3.370000e+14                    1                    253020000.0   
8.823971e+14                    1                    869820000.0   

               Total Traffic (Bytes)  
MSISDN/Number                         
3.360100e+10            8.786906e+08 

In [25]:
#determine which cluster has the lowest engagement based on the average session duration and the total traffic
cluster_avg = user_engagement.groupby('CLuster').mean()
print(cluster_avg)


         Sessions Frequency  Average Session Duration (ms)  \
CLuster                                                      
0                  1.139775                   6.901807e+07   
1                  1.109058                   1.903887e+08   
2                  2.740365                   1.021163e+08   

         Total Traffic (Bytes)  
CLuster                         
0                 5.281817e+08  
1                 5.290756e+08  
2                 1.524500e+09  


In [26]:
# # plot the clusters
# fig = px.scatter_3d(user_engagement, x='Sessions Frequency', y='Average Session Duration (ms)', z='Total Traffic (Bytes)', color='CLuster')
# fig.show()  

# plot the clusters

In [28]:
#fit k-means with the selected number of clusters 
kmeans = KMeans(n_clusters=3, random_state=42)


#determine which cluster has the lowest engagement based on the average session frequency and the total traffic
less_engaged_cluster = user_engagement.groupby('CLuster')['Total Traffic (Bytes)'].mean().index[0]

#calculate Eucledean distance between each centroides
distances = pairwise_distances_argmin_min(normalized_engagment, kmeans.cluster_centers_)[1]
#reset index
user_engagement = user_engagement.reset_index(inplace=True)

# Assign engagagment scores based on less engaged cluster
user_engagement['Engagement Score'] = distances if user_engagement['CLuster'] == less_engaged_cluster else 1 - distances

print(user_engagement)





AttributeError: 'NoneType' object has no attribute 'groupby'